Research Questions Notes: 
would we have enough vine results? compare against vine reulsts, scale against unhelpful, not helpful, fake
would also have to come up with why vine vs nonvine is an important research question

Readme File:
create a readme, don't submit all the separate files to CMS just to github and creat a readme about what is happening
very tough to look for fake reviews, is there or is there not fake reviews??

Meeting with Professor:
 - project could lean heavily towards text analysis
     - major challenge would be --> how do i turn this block of text into numerical features? 
         - start by counting individual distinct words
     - just looking at factors (e.g. verified purchase, length of review, prescense/absence of title) will be enough for phase 2
     - if struggling --> have a conversation w/ professor again about implementing text analysis
         - **professor could give us a different dataset and compare two different review datasets to see rating pracgices between different online communities, why are there distinctive differences, predict rating of product on another marketplace that has diff marketplace standard**
 - looking for relationships between data, how we can build a model to proedict other target vairables of interest, 
 - research question: 
     - finding fake reviews
     - (as first step, exploratory analysis) characterizing different product categories by distribution of review ratings, types
     
using vine reviews as a gold standard while subsetting 10,000 reviews, we create can two distinct datasets two prevent loss of vine reviews:
 - take all the vine reviews (careful not to amplify by making sure to distinguish it as a vine review) 
 - second dataset of non vine reviews that is randomly sampled

# Research Question
1. Are Amazon Beauty product reviews an accurate representation of how well a brand is doing in the greater market? Are we able to determine general consumer sentiment about a brand based on just Amazon reviews?

In [1]:
import pandas as pd

# Data Collection and Cleaning of Amazon Review Datasets

In [2]:
beauty_raw = pd.read_csv('/Users/michellehui/info2950_project/amazon_reviews_us_Beauty_v1_00.tsv', sep = '\t', error_bad_lines=False)

b'Skipping line 10093: expected 15 fields, saw 22\nSkipping line 31965: expected 15 fields, saw 22\nSkipping line 49886: expected 15 fields, saw 22\nSkipping line 49905: expected 15 fields, saw 22\n'
b'Skipping line 67579: expected 15 fields, saw 22\nSkipping line 75367: expected 15 fields, saw 22\nSkipping line 92462: expected 15 fields, saw 22\nSkipping line 105041: expected 15 fields, saw 22\nSkipping line 109697: expected 15 fields, saw 22\nSkipping line 121931: expected 15 fields, saw 22\n'
b'Skipping line 139492: expected 15 fields, saw 22\nSkipping line 158729: expected 15 fields, saw 22\nSkipping line 165784: expected 15 fields, saw 22\nSkipping line 176996: expected 15 fields, saw 22\nSkipping line 182928: expected 15 fields, saw 22\nSkipping line 195841: expected 15 fields, saw 22\n'
b'Skipping line 196938: expected 15 fields, saw 22\nSkipping line 202535: expected 15 fields, saw 22\nSkipping line 261147: expected 15 fields, saw 22\n'
b'Skipping line 265777: expected 15 field

b'Skipping line 1979093: expected 15 fields, saw 22\nSkipping line 1982997: expected 15 fields, saw 22\nSkipping line 1992924: expected 15 fields, saw 22\nSkipping line 1996161: expected 15 fields, saw 22\nSkipping line 2003175: expected 15 fields, saw 22\nSkipping line 2024153: expected 15 fields, saw 22\nSkipping line 2026345: expected 15 fields, saw 22\n'
b'Skipping line 2041159: expected 15 fields, saw 22\nSkipping line 2042954: expected 15 fields, saw 22\nSkipping line 2044244: expected 15 fields, saw 22\nSkipping line 2047949: expected 15 fields, saw 22\nSkipping line 2051022: expected 15 fields, saw 22\nSkipping line 2052365: expected 15 fields, saw 22\nSkipping line 2064460: expected 15 fields, saw 22\nSkipping line 2077010: expected 15 fields, saw 22\nSkipping line 2083893: expected 15 fields, saw 22\n'
b'Skipping line 2097514: expected 15 fields, saw 22\nSkipping line 2100479: expected 15 fields, saw 22\nSkipping line 2103183: expected 15 fields, saw 22\nSkipping line 2108608

In [3]:
beauty_raw.to_csv('amazonreviews_beauty_raw.csv',index=False)

In [4]:
beauty_raw = pd.read_csv('amazonreviews_beauty_raw.csv')

In [5]:
beauty = beauty_raw.copy()

Here we converted the review date to datetime type. When then kept only the reviews written in 2015, which is the most recent year in the dataset. We chose 2015 so that we are able to compare against our 2015 market share data.

In [6]:
# keeping reviews written in 2015
beauty['review_date'] = pd.to_datetime(beauty['review_date'])
beauty = beauty.loc[beauty['review_date'].dt.year == 2015]

As Amazon sells many smaller, indie brands, we are filtering for the select corporate brands that we will be analyzing. The five leading companies we will be looking at are L'oreal, Coty, Revlon, Johnson and Johnson, and Proctor and Gamble. We are including the leading brands that they own as additional brands to filter for.

In [7]:
loreal = ["L'oreal", 'Garnier', 'Maybelline', 'Nyx', 'Cerave']
coty = ['Rimmel', 'Covergirl', 'Sally Hansen']
revlon = ['Revlon', 'Almay']
jnj = ['Clean & Clear', 'Neutrogena', 'Aveeno', 'OGX', 'Lubriderm'] 
png = ['Olay','Herbal Essences', 'Pantene', 'Head and Shoulders','Aussie']
brands = loreal + coty + revlon + jnj
beauty = beauty.loc[beauty['product_title'].str.contains('|'.join(brands))]

In [8]:
# add columns by parent corporation
beauty.loc[beauty['product_title'].str.contains('|'.join(loreal)), 'corporation'] = "L'oreal"
beauty.loc[beauty['product_title'].str.contains('|'.join(coty)), 'corporation'] = "Coty"
beauty.loc[beauty['product_title'].str.contains('|'.join(revlon)), 'corporation'] = "Revlon"
beauty.loc[beauty['product_title'].str.contains('|'.join(jnj)), 'corporation'] = "Johnson & Johnson"
beauty.loc[beauty['product_title'].str.contains('|'.join(png)), 'corporation'] = "Proctor & Gamble"
beauty.corporation.unique()

array(["L'oreal", 'Revlon', 'Coty', 'Johnson & Johnson',
       'Proctor & Gamble'], dtype=object)

In order to narrow the dataset down to a more reasonable size, we are randomly selecting 40,000 observations. We then reset the index on each dataset to be from 0-39,999.

In [9]:
# randomly narrowing down to 40,000
beauty = beauty.sample(n=40000).reset_index()

Here we are dropping the unecessary columns.
 - We first dropped the Marketplace column because all the products were sold in the US.
 - We also dropped the index column as we reset the index. 
 - We dropped product_category as all products fall into Beauty
 - total_votes is the same as helpful_votes and is redundant

In [10]:
# dropping marketplace, index column, product category, total votes
beauty = beauty.drop(columns= ['marketplace','index','product_category', 'total_votes'], axis=1)

In [11]:
# downloading cleaned csv file
beauty.to_csv('amazonreviews_beauty.csv',index=False)

In [12]:
# importing cleaned amazon review datasets
beauty_clean = pd.read_csv('amazonreviews_beauty.csv')

# Data Description

## Motivation
 - **Why was this dataset created?**
     - This dataset was intended to facilitate public study into the properties of Amazon customer reviews including how consumers evolve, express, and evaluate Amazon products.
 
 - **Who funded the creation of the dataset?**
     - Amazon funded and collected this Amazon Customer Reviews Product for public use. The information is publicly available on Amazon Web Services.

## Composition
  - **What are the observations (rows) and the attributes (columns)?**
      - The observations consist of each individual review. Each review has the following column attributes: customer_id, review_id, product_id, product_title, star_rating (Scale of 1-5), helpful_votes, vine (Amazon invites select credible users to review products through their Vine program), verified_purchase, review_headline, review_body, and review_date.
  
 - **How many instances are there in the dataset? Does it contain all possible instances or is it a sample? What processes might have influenced what data was observed and recorded and what was not?**
     - There are a total of 40,000 observations. The original dataset with over a million reviews in the beauty category is a sample collected by Amazon. Amazon constructed this dataset hoping to represent a sample of different customer opinions and demographics. We then narrowed the dataset even further by most recent year and then by select brands leading the beauty/hair care industry. 
 
 - **Where can your raw source data be found, if applicable? Provide a link to the raw data (hosted in a Cornell Google Drive or Cornell Box).**
     - The raw source can be found on Amazon Web Services. Here is the link to raw datasets also uploaded on google drive: https://drive.google.com/drive/folders/1F1Z1ERhp18uaB9J1d-dkGYhBCbJnPyXC?usp=sharing
 
 - **Is the data self contained or does it rely on other outside websites?**
     - The data is self-contained in a downloaded tsv file.
 
 - **Does this data relate to individuals? Can the individuals be identified? Are there any subpopulations that can be identified?**
     - Each individual review is associated to a reviewer through customer_id. However, there is no further information about this individual and customers cannot be split off into smaller subpopulations by demographics.
 
## Collection Process
 - **If people are involved, were they aware of the data collection and if so, what purpose did they expect the data to be used for?**
     - The reviewers in the dataset are not aware of their reviews being used for this purpose. They gave their consent to Amazon when they published their reviews publicly on Amazon. 
 
 - **What time frame was the data collected?**
     - Amazon collected reviews starting from 1995 all the way to 2015. 
 
## Preprocessing
 - **What preprocessing was done, and how did the data come to be in the form that you are using?**
     - We conducted filtering on certain column values by year and product brand. We then randomly selected 40,000 observations to narrow down the size of our dataset.
 
  - **Are there are any errors, sources of noise, or redundancies in the data?**
      - There were some observations with additional attributes/columns when parsing in the data from a csv file. We handled this by skipping over these lines and not including in the final dataset as it would not be reasonable to comb through the millions of reviews to individually handle each of these errors.
 
## Uses
 - **Has this data been used before? If so, what for?**
     - There are no public publishings using this dataset that could be found. However, we found a few other Amazon review analysis using other Amazon datasets. These analyses consisted mainly of review sentiment analysis. 

# Data Limitations
 - Amazon has a lot of other smaller, indie brands that we have excluded from the cleaned dataset. Because we are not including them, our analysis might not accurately reflect the whole market share and beauty space.
 - It could be difficult to analyze the review text content as it requires in-depth natural language processing, which is beyond the scope of what we will learn in this course. 
 - By selecting only 40,000 reviews, we also risk losing a lot of important observations such as vine reviews, reviews that are voted highly helpful, and various reviews across products that could have very beneficial content.

# Exploratory Data Analysis
 ## Amazon
 Sydney:
 - histogram of star ratings by parent company vs overall distribution
     - summary statistics: mean, median, standard deviation
 - historgram of review length by parent company vs overall distribution
     - summary statistics
 - correlation between review length and star ratings
     - correlation coefficients
 - market share graphs
 - writing 
     
 Michelle:    
 - bar chart (bins of helpful votes) to see distribution of helpful votes
 - percentage of verified purchases, vine reviews by parent company
     - correlation between verified purchases and helpful votes
     - correlation between vine and helpful votes
 - distribution of vine, verified purchases, etc. ratings compare
 - markdown and cleaning of data

# Question for Reviewers
1. Are we able to make predictions from observed trends in the Amazon beauty reviews? If so, what types of predictions will we be able to reliably make?
2. Are there any other research areas that we can explore?
3. Should we attempt to conduct natural language processing/sentiment analysis on the review text? Will that be beyond our capabilities as introductory students.
4. Should we expand on into new product categories/corporations to get a more well-rounded overview of consumer review trends?